# Домашнее задание №9. Языковое моделирование

In [8]:
import tensorflow as tf
from keras.callbacks import EarlyStopping
import numpy as np
import os
import time

## Заданиe
Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов

## План решения

[0. Загрузка и просмотр данных](#section_0)

[1. Модель генерации текста](#section_1)

[1.1. Подготовка данных](#section_1.1)

[1.2. Построение модели](#section_1.2)

[1.3. Обучение модели](#section_1.3)

[2. Генерация текста](#section_2)

## 0. Загрузка и просмотр данных  <a id='section_0'></a>

In [3]:
#загружаем файл
text = open('evgenyi_onegin.txt', 'rb').read().decode(encoding='utf-8')

# Определяем длину текста (количество букв в нем)
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [4]:
#просматриваем фрагмент текста
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [5]:
# уникальные буквы в файле
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [6]:
#словарь из букв
vocab

['\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'V',
 'W',
 'X',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y',
 'z',
 '{',
 '}',
 'А',
 'Б',
 'В',
 'Г',
 'Д',
 'Е',
 'Ж',
 'З',
 'И',
 'К',
 'Л',
 'М',
 'Н',
 'О',
 'П',
 'Р',
 'С',
 'Т',
 'У',
 'Ф',
 'Х',
 'Ц',
 'Ч',
 'Ш',
 'Ь',
 'Э',
 'Ю',
 'Я',
 'а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ж',
 'з',
 'и',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'ф',
 'х',
 'ц',
 'ч',
 'ш',
 'щ',
 'ъ',
 'ы',
 'ь',
 'э',
 'ю',
 'я']

In [9]:
# Задаем отображения из множества уникальных букв в множество индексов
char2idx = {u:i for i, u in enumerate(vocab)}  #{уникальная буквы : ее индекс из словаря}

# Представление текста в виде последовательности чисел (индексы букв из словаря)
idx2char = np.array(vocab) # список из символов словаря (можно по индексу извлекать букву)
text_as_int = np.array([char2idx[c] for c in text])

In [10]:
print(text[:500]), print(text_as_int[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо
[ 71 110 104 109 116  99 112 103 115   1  87 104 115 102 104 104 101 107
 122   1  85 118 123 109 107 112   0   0   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1  76 101 102 104 112 107 108   1  84 112 104 102
 107 112   0   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1  86
 113 111  99 112   1 101   1 116 117 107 120  99 120   0   0 

(None, None)

## 1. Модель генерации текста  <a id='section_1'></a>

### 1.1. Подготовка данных <a id='section_1.1'></a>

In [35]:
# Максимальная длина предложения для входных данных (в буквах)
seq_length = 100

#количество эпох
examples_per_epoch = len(text)//(seq_length+1)

In [36]:
# создаем датасет из данных
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
for element in char_dataset:
    print(len(char_dataset))
    print(element)
    break

286984
tf.Tensor(71, shape=(), dtype=int64)


In [37]:
for element in char_dataset:
    print(idx2char[element.numpy()])
    break

А


In [38]:
#создаем батчи из seq_length+1 элементов
sequences = char_dataset.batch(seq_length+1, drop_remainder=True) #drop_remainder удаляет неполный батч (с длиной менее seq_length+1)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [39]:
# Разбиваем каждый батч на признаки и целевую переменную (последнюю букву)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

#применяем функцию split_input_target ко всем батчам
dataset = sequences.map(split_input_target)

In [40]:
for input_example, target_example in dataset:
    print(f'Количество батчей: {len(dataset)}')
    print(f'Данные: {input_example}')
    print(f'Целевая переменная: {target_example}')
    break

Количество батчей: 2841
Данные: [ 71 110 104 109 116  99 112 103 115   1  87 104 115 102 104 104 101 107
 122   1  85 118 123 109 107 112   0   0   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1  76 101 102 104 112 107 108   1  84 112 104 102
 107 112   0   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1]
Целевая переменная: [110 104 109 116  99 112 103 115   1  87 104 115 102 104 104 101 107 122
   1  85 118 123 109 107 112   0   0   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1  76 101 102 104 112 107 108   1  84 112 104 102 107
 112   0   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1]


In [41]:
for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


### 1.2. Построение модели <a id='section_1.2'></a>

In [42]:
# размер батча
BATCH_SIZE = 64

# размер буфера для перемешивания данных
BUFFER_SIZE = 10000

# перемешивание разделенных данных
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [43]:
# Длина словаря в буквах
vocab_size = len(vocab)

# Размерность эмбеддинга
embedding_dim = 256

# Число ячеек
rnn_units = 1024

In [44]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [45]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [46]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [47]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           33536     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense (Dense)               (64, None, 131)           134275    
                                                                 
Total params: 30,592,899
Trainable params: 30,592,899
No

In [48]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 131), dtype=float32, numpy=
array([[-3.81107156e-06, -2.06065379e-05,  6.71569887e-06, ...,
         1.42809695e-05,  5.45804687e-06, -1.44679689e-07],
       [-9.50220056e-06, -4.80803355e-05,  2.69599659e-05, ...,
         2.99920703e-05,  1.01583400e-05,  8.11330665e-06],
       [-1.52972734e-05, -5.57256208e-05,  8.76246777e-05, ...,
         4.71799140e-05,  2.90016451e-05,  4.86779172e-05],
       ...,
       [-2.63679004e-03,  1.84492106e-04,  3.41221457e-03, ...,
        -2.57675396e-03, -1.28943566e-03,  1.89420162e-03],
       [-2.65124207e-03,  2.40691399e-04,  3.25254072e-03, ...,
        -2.80535407e-03, -1.57234259e-03,  1.79659470e-03],
       [-2.68921396e-03,  3.05590627e-04,  3.12503008e-03, ...,
        -3.01102805e-03, -1.84767391e-03,  1.70347968e-03]], dtype=float32)>

In [49]:
#индексы букв (извлекаем выборки из категориального распределения)
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1) #количество независимых выборок 1

#убираем лишнюю размерность (список индексов)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([118,  11,  14,  29,  94,  82,  44,  81,  76,  95,  88,  18,  69,
       107,  80,  32,   7, 107,  72, 107,  21,  88,  12,  29,  40, 109,
        27,  15,  54,  61,  24,  62, 123, 122,  90,  15, 112,  39,   5,
       115,   4, 120,  61,  18,  47,  84,  56, 111,   6, 123,  21,  72,
       100, 112,  92,  75,  91,  77,  50, 101,  40,   2, 108,  52,  27,
       102,  29,  57,  74,  60,  29,  33, 104,  39,  32,  59, 113,   6,
       109,  17,  64,  21, 103, 127,  14,  93,   8,  85,  40,  34,  33,
       104,  72,  64, 108, 106, 118,  42, 111,  38])

In [50]:
#входной пример
#возвращаем строку
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()

#предсказанная буква
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ' Позвольте мне, читатель мой,\n                        Заняться старшею сестрой.\n\n                   '

Next Char Predictions: 
 "у14GШМYЛЕЬТ8{иКL,иБи;Т2GTкE5krBsшчФ5нS(р'хr8cОmм)ш;БбнЦДХЖfвT!йhEгGnГqGMеSLpо)к7u;дь4Ч-ПTNMеБuйзуWмR"


### 1.3. Обучение модели <a id='section_1.3'></a>

In [51]:
#функция потерь
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.8751607


In [52]:
#компиляция модели
model.compile(optimizer='adam', loss=loss)

In [53]:
# место для хранения checkpoint
checkpoint_dir = './training_checkpoints'
# Имя файла checkpoint
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=44*100, # сохраняем каждую сотую эпоху - в эпохе 44 батча (это видно при запуске обучения)
    save_weights_only=True,
    )

In [54]:
#обучение модели
EPOCHS = 100
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
44/44 [==============================] - 20s 325ms/step - loss: 2.5070
Epoch 2/100
44/44 [==============================] - 15s 332ms/step - loss: 2.2259
Epoch 3/100
44/44 [==============================] - 15s 344ms/step - loss: 1.9836
Epoch 4/100
44/44 [==============================] - 16s 353ms/step - loss: 1.7745
Epoch 5/100
44/44 [==============================] - 16s 363ms/step - loss: 1.7089
Epoch 6/100
44/44 [==============================] - 16s 363ms/step - loss: 1.5962
Epoch 7/100
44/44 [==============================] - 16s 358ms/step - loss: 1.5279
Epoch 8/100
44/44 [==============================] - 16s 351ms/step - loss: 1.4836
Epoch 9/100
44/44 [==============================] - 16s 356ms/step - loss: 1.4369
Epoch 10/100
44/44 [==============================] - 16s 357ms/step - loss: 1.4048
Epoch 11/100
44/44 [==============================] - 16s 363ms/step - loss: 1.3884
Epoch 12/100
44/44 [==============================] - 16s 356ms/step - loss: 1.3701
E

In [59]:
#предсказание модели
example_batch_predictions = model(input_example_batch)
print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [60]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [61]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ' Позвольте мне, читатель мой,\n                        Заняться старшею сестрой.\n\n                   '

Next Char Predictions: 
 ' ртнольте пне, митатель гой,\n                        Ду явься старшею сестрой.\n\n                    '


### 2. Генерация текста <a id='section_2'></a>

In [62]:
#Находим имя файла последней сохраненной контрольной точки
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_100'

In [63]:
#строим модель
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

#загружаем веса из последней сохраненной контрольной точки в модель
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [64]:
#информация о модели
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            33536     
                                                                 
 lstm_4 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 lstm_5 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_6 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_7 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 dense_1 (Dense)             (1, None, 131)            134275    
                                                                 
Total params: 30,592,899
Trainable params: 30,592,899


In [71]:
def generate_text(model, start_string):
    # Этап оценки (генерация текста с использованием обученной модели)

    # число букв для генераци
    num_generate = 100

    # Преобразование начальной строки в числа (векторизация)
    input_eval = [char2idx[s] for s in start_string]
    #Возвращаем тензор с осью длины 1, вставленной первой в индекс
    input_eval = tf.expand_dims(input_eval, 0)

    # Пустая строка для хранения результатов
    text_generated = []

    # Низкая температура приводит к более предсказуемому тексту.
    # Более высокая температура приводит к более неожиданному тексту.
    temperature = 0.001

    # здесь batch size == 1
    # сбрасываем состояния всех слоев в модели
    model.reset_states()
    for i in range(num_generate):
        #получаем предсказания модели
        predictions = model(input_eval)
        #удаляем первую размерность в предсказании
        predictions = tf.squeeze(predictions, 0)
        # использование категориального распределения для прогнозирования символа, возвращаемого моделью
        predictions = predictions / temperature
        #извлекаем выборку из категориального распределения
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Передаем предсказанный символ в качестве следующего ввода в модель
        # вместе с предыдущим скрытым состоянием 
        # добавляем 1 первым индексом к размерности
        input_eval = tf.expand_dims([predicted_id], 0)

        #сохраняем предсказанную букву
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [74]:
text_ = generate_text(model, start_string=u"Вот он идет ")
print(text_)

Вот он идет был небес,
                        И в молчаливом кабинете,
                        И в молчаливом к
